# Imports

In [16]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, exp
from biogeme.models import loglogit
from biogeme.tools import likelihood_ratio_test
from biogeme.results import compile_estimation_results
from biogeme.models import loglogit,  boxcox
from biogeme.models.piecewise import piecewise_formula
from biogeme.models import lognested
from biogeme.nests import OneNestForNestedLogit, NestsForNestedLogit
from biogeme.biogeme import BIOGEME

import pickle

import numpy as np
import os

from scipy.stats import chi2


# Data & Variables

In [17]:
# Define the relative path to the data folder
file_path = os.path.join(os.pardir, 'lpmc01.dat')

#file_path = os.path.join(data_folder, 'lpmc01.dat')

df = pd.read_csv(file_path, sep = '\t')
df['age_scaled'] = (df['age'] - df['age'].mean()) / df['age'].std()
df['cost_driving'] = df['cost_driving_ccharge'] + df['cost_driving_fuel']
df['dur_pt'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_int'] + df['dur_pt_bus']

database1 = db.Database('lpmc01', df)


# Define the given veriables 
dur_pt = Variable('dur_pt')
cost_driving = Variable('cost_driving')
age_scaled = Variable('age_scaled')
trip_id = Variable('trip_id')
household_id = Variable('household_id')
person_n = Variable('person_n')
trip_n = Variable('trip_n')
travel_mode = Variable('travel_mode')
purpose = Variable('purpose')
fueltype = Variable('fueltype')
faretype = Variable('faretype')
bus_scale = Variable('bus_scale')
survey_year = Variable('survey_year')
travel_year = Variable('travel_year')
travel_month = Variable('travel_month')
travel_date = Variable('travel_date')
day_of_week = Variable('day_of_week')
start_time = Variable('start_time')
age = Variable('age')
female = Variable('female')
driving_license = Variable('driving_license')
car_ownership = Variable('car_ownership')
distance = Variable('distance')
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
dur_pt_access = Variable('dur_pt_access') # Predicted total access and egress time for public transport route in hours
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int') # Time taken (hrs) at each interchange point
pt_interchanges = Variable('pt_interchanges')   # Number of interchange points in public transport route
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
cost_driving_fuel = Variable('cost_driving_fuel')   # Estimated fuel cost of driving route in GBP
cost_driving_ccharge = Variable('cost_driving_ccharge')  # Estimated congestion charge cost of driving route in GBP
driving_traffic_percent = Variable('driving_traffic_percent')


variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")



# Define pt_cost (not needed)
# Original paper, page 31: "Public transport fares are determined for single trips using Oystercard/contactless payment."
# Therefore, cost_transit should already consider faretype and bus_scale

database = db.Database('lpmc01', df)
variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")

# Define driving cost
cost_driving = cost_driving_ccharge + cost_driving_fuel

# Define time taken by each mode of transport
dur_pt = dur_pt_access + dur_pt_int + dur_pt_bus + dur_pt_rail  # Public transport (external) time 

'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.
'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.


In [18]:
all_results = {}

# Model A

## Definition

In [19]:
# Assume every mode of transport is available
availability_walk = 1  
availability_cycle = 1  
availability_pt = 1     
availability_drive = 1

availability = {
    1: availability_walk,   # Walking
    2: availability_cycle,  # Cycling
    3: availability_pt,     # Public Transport
    4: availability_drive   # Driving
}

In [20]:
ASC_CYCLE = Beta('ASC_CYCLE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_DRIVE = Beta('ASC_DRIVE', 0, None, None, 0)

# Define generic parameters for cost and travel time
B_COST = Beta('B_COST', 0, None, None, 0)

# Define interaction terms between ASCs and age_scaled
#ASC_CYCLE_AGE = ASC_CYCLE + Beta('ASC_CYCLE_AGE', 0, None, None, 0) * age_scaled
#ASC_PT_AGE = ASC_PT + Beta('ASC_PT_AGE', 0, None, None, 0) * age_scaled
ASC_DRIVE_AGE =  Beta('ASC_DRIVE_AGE', 0, None, None, 0) 
ASC_WALK_AGE =  Beta('ASC_WALK_AGE', 0, None, None, 0)

B_TIME_WALK = Beta('B_TIME_WALK', 0, None, None, 0)
B_TIME_CYCLE = Beta('B_TIME_CYCLE', 0, None, None, 0)
B_TIME_PT = Beta('B_TIME_PT', 0, None, None, 0)
B_TIME_DRIVE = Beta('B_TIME_DRIVE', 0, None, None, 0)


# Utility functions with interaction terms
V_WALK = ASC_WALK_AGE * age_scaled + B_TIME_WALK * dur_walking
V_CYCLE = ASC_CYCLE + B_TIME_CYCLE * dur_cycling
V_PT = ASC_PT + B_COST * cost_transit + B_TIME_PT * dur_pt
V_DRIVE = (ASC_DRIVE + ASC_DRIVE_AGE * age_scaled) + B_COST * cost_driving + B_TIME_DRIVE * dur_driving

# Specify the model
model_2a = loglogit({1: V_WALK, 2: V_CYCLE, 3: V_PT, 4: V_DRIVE}, availability, travel_mode)

## Results

In [21]:
# Estimate Model 2 - Specification 1
biogeme_spec1 = bio.BIOGEME(database, model_2a)
biogeme_spec1.modelName = "Model_2A"
results_m2a = biogeme_spec1.estimate()

all_results['Model_2A'] = results_m2a

# Display estimation results
print("Estimation results for Model 2A:")
print(results_m2a.get_estimated_parameters())

Estimation results for Model 2A:
                  Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CYCLE     -4.611915      0.199423   -23.126239      0.000000
ASC_DRIVE     -2.142069      0.147738   -14.499147      0.000000
ASC_DRIVE_AGE  0.117403      0.035173     3.337894      0.000844
ASC_PT        -2.609414      0.148477   -17.574522      0.000000
ASC_WALK_AGE  -0.178174      0.048092    -3.704877      0.000211
B_COST        -0.182629      0.017644   -10.350876      0.000000
B_TIME_CYCLE  -6.457473      0.483126   -13.366019      0.000000
B_TIME_DRIVE  -6.556007      0.378364   -17.327234      0.000000
B_TIME_PT     -3.481130      0.242150   -14.375936      0.000000
B_TIME_WALK   -9.119739      0.464578   -19.630138      0.000000


In [22]:
print(results_m2a.print_general_statistics())

Number of estimated parameters:	10
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4202.68
Likelihood ratio test for the init. model:	5457.583
Rho-square for the init. model:	0.394
Rho-square-bar for the init. model:	0.392
Akaike Information Criterion:	8425.361
Bayesian Information Criterion:	8490.533
Final gradient norm:	5.0907E-02
Nbr of threads:	16



## Testing Against Model 1

In [23]:
folder_path = os.path.join(os.pardir, 'Model_1')
file_path = os.path.join(folder_path, 'Model_1.pickle')

# open a file, where you stored the pickled data
file = open(file_path, 'rb')

# dump information to that file
results_m1 = pickle.load(file)

# close the file
file.close()

In [24]:
loglikehood_m2a = results_m2a.data.logLike
num_params_m2a = results_m2a.data.nparam

loglikehood_m1 = results_m1.logLike
num_params_m1 = results_m1.nparam

# Calculate the LR statistic
LR = 2 * (loglikehood_m2a - loglikehood_m1)

# Degrees of freedom
df = num_params_m2a - num_params_m1

# Critical value at 0.05 significance level
critical_value = chi2.ppf(0.95, df)

print("Likelihood Ratio:", LR)
print("Degrees of Freedom:", df)
print("Critical Chi-Square Value (0.05 significance):", critical_value)

if LR > critical_value:
    print("Model 2a is significantly better than Model 1.")
else:
    print("No significant improvement in Model 2a over Model 1.")



Likelihood Ratio: 42.61134228184892
Degrees of Freedom: 2
Critical Chi-Square Value (0.05 significance): 5.991464547107979
Model 2a is significantly better than Model 1.


# Model B

## Definition

In [25]:
# Define alternative-specific parameters for travel time
B_TIME_WALK = Beta('B_TIME_WALK', 0, None, None, 0)
B_TIME_CYCLE = Beta('B_TIME_CYCLE', 0, None, None, 0)
B_TIME_PT = Beta('B_TIME_PT', 0, None, None, 0)
B_TIME_DRIVE = Beta('B_TIME_DRIVE', 0, None, None, 0)

ASC_CYCLE = Beta('ASC_CYCLE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_DRIVE = Beta('ASC_DRIVE', 0, None, None, 0)

# Define generic parameters for cost and travel time
B_COST = Beta('B_COST', 0, None, None, 0)

B_TIME_WALK_AGE = Beta('B_TIME_WALK_AGE', 0, None, None, 0)
B_TIME_CYCLE_AGE = Beta('B_TIME_CYCLE_AGE', 0, None, None, 0) 
B_TIME_PT_AGE = Beta('B_TIME_PT_AGE', 0, None, None, 0)
#B_TIME_DRIVE_AGE = Beta('B_TIME_DRIVE_AGE', 0, None, None, 0) * age_scaled

# Updated utility functions with age interaction for travel time
V_WALK = (B_TIME_WALK + B_TIME_WALK_AGE * age_scaled) * dur_walking
V_CYCLE = ASC_CYCLE + (B_TIME_CYCLE + B_TIME_CYCLE_AGE* age_scaled) * dur_cycling
V_PT = ASC_PT + B_COST * cost_transit + (B_TIME_PT + B_TIME_PT_AGE* age_scaled) * dur_pt
V_DRIVE = ASC_DRIVE + B_COST * cost_driving + (B_TIME_DRIVE) * dur_driving

# Associate utility functions with the mode choice
V = {
    1: V_WALK,    # Walking
    2: V_CYCLE,   # Cycling
    3: V_PT,      # Public Transport
    4: V_DRIVE    # Driving
}

# Specify the model
model_2b = loglogit({1: V_WALK, 2: V_CYCLE, 3: V_PT, 4: V_DRIVE}, availability, travel_mode)

## Results

In [26]:
# Estimate Model 2 - Specification 1
biogeme_spec1 = bio.BIOGEME(database, model_2b)
biogeme_spec1.modelName = "Model_2B"
results_m2b = biogeme_spec1.estimate()

all_results['Model_2B'] = results_m2b

# Display estimation results
print("Estimation results for Model 2b:")
print(results_m2b.get_estimated_parameters())

Estimation results for Model 2b:
                     Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CYCLE        -4.617556      0.200491   -23.031189  0.000000e+00
ASC_DRIVE        -2.154812      0.149765   -14.387947  0.000000e+00
ASC_PT           -2.614215      0.150143   -17.411540  0.000000e+00
B_COST           -0.184054      0.017812   -10.333033  0.000000e+00
B_TIME_CYCLE     -6.553376      0.485295   -13.503888  0.000000e+00
B_TIME_CYCLE_AGE -0.639246      0.231275    -2.764007  5.709634e-03
B_TIME_DRIVE     -6.599087      0.379026   -17.410667  0.000000e+00
B_TIME_PT        -3.535441      0.240127   -14.723232  0.000000e+00
B_TIME_PT_AGE    -0.409863      0.072999    -5.614624  1.969899e-08
B_TIME_WALK      -9.195549      0.478606   -19.213202  0.000000e+00
B_TIME_WALK_AGE  -0.719438      0.158845    -4.529190  5.921014e-06


In [27]:
print(results_m2b.print_general_statistics())

Number of estimated parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4194.341
Likelihood ratio test for the init. model:	5474.262
Rho-square for the init. model:	0.395
Rho-square-bar for the init. model:	0.393
Akaike Information Criterion:	8410.681
Bayesian Information Criterion:	8482.37
Final gradient norm:	1.9900E-02
Nbr of threads:	16



## Testing Against Model 1

In [28]:
loglikehood_m2b = results_m2b.data.logLike
num_params_m2b = results_m2b.data.nparam

# Calculate the LR statistic
LR = 2 * (loglikehood_m2b - loglikehood_m1)

# Degrees of freedom
df = num_params_m2a - num_params_m1

# Critical value at 0.05 significance level
critical_value = chi2.ppf(0.95, df)

print("Likelihood Ratio:", LR)
print("Degrees of Freedom:", df)
print("Critical Chi-Square Value (0.05 significance):", critical_value)

if LR > critical_value:
    print("Model 2b is significantly better than Model 1.")
else:
    print("No significant improvement in Model 2b over Model 1.")



Likelihood Ratio: 59.290726222418016
Degrees of Freedom: 2
Critical Chi-Square Value (0.05 significance): 5.991464547107979
Model 2b is significantly better than Model 1.


# Comparing Model A and Model B

In [29]:
comparison_table, _ = compile_estimation_results(all_results)
display(comparison_table)

,Model_2A,Model_2B
Number of estimated parameters,10,11
Sample size,5000,5000
Final log likelihood,-4202.680286,-4194.340594
Akaike Information Criterion,8425.360573,8410.681189
Bayesian Information Criterion,8490.532505,8482.370314
ASC_CYCLE (t-test),-4.61 (-23.1),-4.62 (-23)
ASC_DRIVE (t-test),-2.14 (-14.5),-2.15 (-14.4)
ASC_DRIVE_AGE (t-test),0.117 (3.34),
ASC_PT (t-test),-2.61 (-17.6),-2.61 (-17.4)
ASC_WALK_AGE (t-test),-0.178 (-3.7),
